In [ ]:
#@title Mount google drive and Install dependencies.
from google.colab import drive
drive.mount('/content/drive')

!pip install bitsandbytes accelerate>=0.16.0 torchvision transformers>=4.25.1 ftfy tensorboard Jinja2

!pip install peft==0.7.0

!pip install git+https://github.com/yinglingyang/diffusers.git

# download SDXL DreamBooth training script
!wget https://raw.githubusercontent.com/yinglingyang/diffusers/main/examples/dreambooth/train_dreambooth_lora_sdxl.py

!accelerate config default

Load training images


In [ ]:
Original_images_Path = "/content/drive/MyDrive/ACV_Final/dataset"
Output_Path = "/content/drive/MyDrive/ACV_Final/model"

image_path = Original_images_Path
output_path = Output_Path

import os
os.makedirs(output_path, exist_ok=True)

!ls "$image_path"


In [ ]:
#@title Training Parameters

prompt = "ACV style of photo"#@param {type:"string"}
learning_rate = 1e-4 #@param {type:"number"}
resolution=512 #@param {type:"integer"}
max_train_steps=800 #@param {type:"integer"}
checkpointing_steps=200 #@param {type:"integer"}

In [ ]:
#@title Start Training

#!/usr/bin/env bash
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
  --pretrained_vae_model_name_or_path="madebyollin/sdxl-vae-fp16-fix" \
  --instance_data_dir=$image_path \
  --output_dir=$output_path \
  --mixed_precision="fp16" \
  --instance_prompt="$prompt" \
  --resolution=$resolution \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --learning_rate=$learning_rate \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --use_8bit_adam \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=$checkpointing_steps \
  --seed="0"

Now you can use model to run inference!